# **Data Book Notebook**

* This section aims to enrich and centralize the necessary data for the project's execution. Some of this data will be created through calculations based on existing data.

## **Initial Setup**

### Install Libs

In [1]:
%pip install scikit-learn --q --no-cache

Note: you may need to restart the kernel to use updated packages.


### Import Packages

In [2]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

### Create a file path default

In [3]:
file_path_cleaned = str(Path(os.getcwd()).parent/"data/cleaned")
file_path_book  = str(Path(os.getcwd()).parent/"data/book")

## **Data Book**

* This section's mission is to enrich the data captured and cleaned by the processes in the notebooks "01_data_aquisition.ipynb" and "02_data_cleaning.ipynb". This enrichment will be achieved through calculations with existing fields and possibly the acquisition of new external data. It's important to emphasize the significance of running the previous notebooks for the proper functioning of the project.

### Fundamentals Book

In [4]:
df_fundamentals_cleaned = pd.read_csv(file_path_cleaned + "/fundamentals_cleaned.csv")
df_fundamentals_cleaned.head()

,ticker,long_name,sector,industry,market_cap,enterprise_value,total_revenue,profit_margins,operating_margins,dividend_rate,...,total_cash,total_cash_per_share,total_debt,earnings_quarterly_growth,revenue_growth,gross_margins,ebitda_margins,return_on_assets,return_on_equity,gross_profits
0,ABCB4.SA,Banco ABC Brasil S.A.,Financial Services,Banks - Regional,4.342988e+09,1.496796e+10,1.941779e+09,0.41576,0.41515,1.56,...,7.774306e+09,35.162,1.829846e+10,0.001,0.003,0.00000,0.00000,0.01530,0.15680,1.973086e+09
1,AGRO3.SA,BrasilAgro - Companhia Brasileira de Proprieda...,Consumer Defensive,Farm Products,2.899807e+09,3.364476e+09,1.249437e+09,0.21492,0.16848,3.24,...,3.838370e+08,3.885,8.720750e+08,6.801,0.671,0.25252,0.21201,0.03839,0.12170,3.155040e+08
2,RAIL3.SA,Rumo S.A.,Industrials,Railroads,4.062026e+10,5.618793e+10,1.031746e+10,0.07638,0.30165,0.07,...,7.656040e+09,4.132,2.184320e+10,3.935,0.121,0.34493,0.43834,0.04252,0.05163,3.146360e+09
3,ALPA3.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.033696e+09,6.280492e+09,4.022153e+09,-0.05671,-0.02786,0.00,...,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09
4,ALPA4.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.046720e+09,6.098251e+09,4.022153e+09,-0.05671,-0.02786,0.00,...,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09


#### Feature Creation

In [5]:
df_fundamentals_book_cols = df_fundamentals_cleaned.copy()

In [6]:
df_fundamentals_book_cols['earnings_growth_rate'] = df_fundamentals_cleaned['earnings_quarterly_growth'] * 100
df_fundamentals_book_cols['dividend_payout_ratio'] = (df_fundamentals_cleaned['trailing_annual_dividend_rate'] / df_fundamentals_cleaned['earnings_quarterly_growth']) * 100
df_fundamentals_book_cols['dividend_payout_ratio'] = df_fundamentals_book_cols['dividend_payout_ratio'].replace([np.inf, -np.inf], np.nan)
df_fundamentals_book_cols['dividend_payout_ratio'] = df_fundamentals_book_cols['dividend_payout_ratio'].apply(lambda x: max(0, x))
df_fundamentals_book_cols['roi'] = df_fundamentals_cleaned['total_revenue'] / df_fundamentals_cleaned['enterprise_value']

* In the code block above, we are working with a DataFrame of company fundamentals. The fields "earnings_growth_rate," "dividend_payout_ratio," and "roi" play pivotal roles in our analysis. "Earnings_growth_rate" informs us about the company's profit growth over time, which is crucial for assessing its potential for appreciation. Meanwhile, "dividend_payout_ratio" aids us in understanding the company's dividend policy, revealing how much of the profits is distributed to shareholders. Lastly, "ROI" is an essential metric for evaluating financial efficiency and company performance in relation to invested capital. These fields are the cornerstones of our analysis and guide our investment decisions.

In [7]:
df_fundamentals_book_cols.columns

Index(['ticker', 'long_name', 'sector', 'industry', 'market_cap',
       'enterprise_value', 'total_revenue', 'profit_margins',
       'operating_margins', 'dividend_rate', 'beta', 'ebitda', 'trailing_pe',
       'forward_pe', 'volume', 'average_volume', 'fifty_two_week_low',
       'fifty_two_week_high', 'price_to_sales_trailing_12_months',
       'fifty_day_average', 'two_hundred_day_average',
       'trailing_annual_dividend_rate', 'trailing_annual_dividend_yield',
       'book_value', 'price_to_book', 'total_cash', 'total_cash_per_share',
       'total_debt', 'earnings_quarterly_growth', 'revenue_growth',
       'gross_margins', 'ebitda_margins', 'return_on_assets',
       'return_on_equity', 'gross_profits', 'earnings_growth_rate',
       'dividend_payout_ratio', 'roi'],
      dtype='object')

In [8]:
df_fundamentals_book_cols.head(2)

,ticker,long_name,sector,industry,market_cap,enterprise_value,total_revenue,profit_margins,operating_margins,dividend_rate,...,earnings_quarterly_growth,revenue_growth,gross_margins,ebitda_margins,return_on_assets,return_on_equity,gross_profits,earnings_growth_rate,dividend_payout_ratio,roi
0,ABCB4.SA,Banco ABC Brasil S.A.,Financial Services,Banks - Regional,4.342988e+09,1.496796e+10,1.941779e+09,0.41576,0.41515,1.56,...,0.001,0.003,0.00000,0.00000,0.01530,0.1568,1.973086e+09,0.1,155000.000000,0.129729
1,AGRO3.SA,BrasilAgro - Companhia Brasileira de Proprieda...,Consumer Defensive,Farm Products,2.899807e+09,3.364476e+09,1.249437e+09,0.21492,0.16848,3.24,...,6.801,0.671,0.25252,0.21201,0.03839,0.1217,3.155040e+08,680.1,47.640053,0.371362


#### Cleaning Missing Values

In [9]:
df_fundamentals_book_missing = df_fundamentals_book_cols.copy()

In [10]:
df_fundamentals_book_missing.isna().sum()

ticker                                0
long_name                             0
sector                                0
industry                              0
market_cap                            0
enterprise_value                      0
total_revenue                         0
profit_margins                        0
operating_margins                     0
dividend_rate                         0
beta                                  0
ebitda                                0
trailing_pe                           0
forward_pe                            0
volume                                0
average_volume                        0
fifty_two_week_low                    0
fifty_two_week_high                   0
price_to_sales_trailing_12_months     0
fifty_day_average                     0
two_hundred_day_average               0
trailing_annual_dividend_rate         0
trailing_annual_dividend_yield        0
book_value                            0
price_to_book                         0


In [11]:
df_fundamentals_book_missing["dividend_payout_ratio"].fillna(0, inplace = True)

In [12]:
df_fundamentals_book_missing.isna().sum()

ticker                               0
long_name                            0
sector                               0
industry                             0
market_cap                           0
enterprise_value                     0
total_revenue                        0
profit_margins                       0
operating_margins                    0
dividend_rate                        0
beta                                 0
ebitda                               0
trailing_pe                          0
forward_pe                           0
volume                               0
average_volume                       0
fifty_two_week_low                   0
fifty_two_week_high                  0
price_to_sales_trailing_12_months    0
fifty_day_average                    0
two_hundred_day_average              0
trailing_annual_dividend_rate        0
trailing_annual_dividend_yield       0
book_value                           0
price_to_book                        0
total_cash               

In [13]:
df_fundamentals_book = df_fundamentals_book_missing.copy()

In [14]:
df_fundamentals_book.head()

,ticker,long_name,sector,industry,market_cap,enterprise_value,total_revenue,profit_margins,operating_margins,dividend_rate,...,earnings_quarterly_growth,revenue_growth,gross_margins,ebitda_margins,return_on_assets,return_on_equity,gross_profits,earnings_growth_rate,dividend_payout_ratio,roi
0,ABCB4.SA,Banco ABC Brasil S.A.,Financial Services,Banks - Regional,4.342988e+09,1.496796e+10,1.941779e+09,0.41576,0.41515,1.56,...,0.001,0.003,0.00000,0.00000,0.01530,0.15680,1.973086e+09,0.1,155000.000000,0.129729
1,AGRO3.SA,BrasilAgro - Companhia Brasileira de Proprieda...,Consumer Defensive,Farm Products,2.899807e+09,3.364476e+09,1.249437e+09,0.21492,0.16848,3.24,...,6.801,0.671,0.25252,0.21201,0.03839,0.12170,3.155040e+08,680.1,47.640053,0.371362
2,RAIL3.SA,Rumo S.A.,Industrials,Railroads,4.062026e+10,5.618793e+10,1.031746e+10,0.07638,0.30165,0.07,...,3.935,0.121,0.34493,0.43834,0.04252,0.05163,3.146360e+09,393.5,1.677255,0.183624
3,ALPA3.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.033696e+09,6.280492e+09,4.022153e+09,-0.05671,-0.02786,0.00,...,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,0.0,0.000000,0.640420
4,ALPA4.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.046720e+09,6.098251e+09,4.022153e+09,-0.05671,-0.02786,0.00,...,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,0.0,0.000000,0.659558


#### Saving Files

In [16]:
Path(file_path_book).mkdir(parents=True, exist_ok=True)
df_fundamentals_book.to_csv(file_path_book + "/fundamentals_book.csv", index=False)

### Macroeconomic Book

In [17]:
df_macroeconomic_cleaned = pd.read_csv(file_path_cleaned + "/macroeconomic_cleaned.csv")
df_macroeconomic_cleaned.head()

,date,selic,confidence,pib,incc,ipca,dolar
0,2019-01-31,6.5,128.64,578214.5,0.49,3.78,3.6513
1,2019-02-28,6.5,139.39,576089.7,0.09,3.89,3.7379
2,2019-03-31,6.5,125.53,601749.8,0.31,4.58,3.8961
3,2019-04-30,6.5,121.71,612918.4,0.38,4.94,3.9447
4,2019-05-31,6.5,117.01,615304.9,0.03,4.66,3.9401


#### Feature Creation

In [18]:
df_macroeconomic_book_cols = df_macroeconomic_cleaned.copy()

In [19]:
df_macroeconomic_book_cols['monthly_inflation'] = df_macroeconomic_book_cols['ipca'].pct_change()
df_macroeconomic_book_cols['gdp_growth'] = df_macroeconomic_book_cols['pib'].pct_change()
df_macroeconomic_book_cols['dollar_growth'] = df_macroeconomic_book_cols['dolar'].pct_change()
df_macroeconomic_book_cols['real_interest_rate'] = df_macroeconomic_book_cols['selic'] - df_macroeconomic_book_cols['ipca']
df_macroeconomic_book_cols['inflation_confidence_difference'] = df_macroeconomic_book_cols['ipca'] - df_macroeconomic_book_cols['confidence']

In [20]:
df_macroeconomic_book_cols.columns

Index(['date', 'selic', 'confidence', 'pib', 'incc', 'ipca', 'dolar',
       'monthly_inflation', 'gdp_growth', 'dollar_growth',
       'real_interest_rate', 'inflation_confidence_difference'],
      dtype='object')

* In the code block above, we are working with a DataFrame of economic indicators. The fields "monthly_inflation," "gdp_growth," "dollar_growth," "real_interest_rate," and "inflation_confidence_difference" are pivotal in our economic analysis. "Monthly_inflation" informs us about the monthly changes in inflation, which is vital for understanding price stability. "GDP_growth" tracks the growth rate of the Gross Domestic Product, indicating the economic health and expansion of the country. "Dollar_growth" reflects changes in the value of the national currency relative to the US dollar, providing insights into exchange rate fluctuations. "Real_interest_rate" measures the interest rate adjusted for inflation, helping us assess borrowing costs and investment opportunities. Lastly, "Inflation_confidence_difference" quantifies the difference in inflation expectations, shedding light on economic confidence levels. These fields are integral to our economic analysis and inform our decisions.

In [21]:
df_macroeconomic_book_cols.head(5)

,date,selic,confidence,pib,incc,ipca,dolar,monthly_inflation,gdp_growth,dollar_growth,real_interest_rate,inflation_confidence_difference
0,2019-01-31,6.5,128.64,578214.5,0.49,3.78,3.6513,NaN,NaN,NaN,2.72,-124.86
1,2019-02-28,6.5,139.39,576089.7,0.09,3.89,3.7379,0.029101,-0.003675,0.023718,2.61,-135.50
2,2019-03-31,6.5,125.53,601749.8,0.31,4.58,3.8961,0.177378,0.044542,0.042323,1.92,-120.95
3,2019-04-30,6.5,121.71,612918.4,0.38,4.94,3.9447,0.078603,0.018560,0.012474,1.56,-116.77
4,2019-05-31,6.5,117.01,615304.9,0.03,4.66,3.9401,-0.056680,0.003894,-0.001166,1.84,-112.35


#### Cleaning MIssing Values

In [22]:
df_macroeconomic_book_missing = df_macroeconomic_book_cols.copy()

In [23]:
df_macroeconomic_book_missing.isna().sum()

date                               0
selic                              0
confidence                         0
pib                                0
incc                               0
ipca                               0
dolar                              0
monthly_inflation                  1
gdp_growth                         1
dollar_growth                      1
real_interest_rate                 0
inflation_confidence_difference    0
dtype: int64

In [24]:
dates = df_macroeconomic_book_missing['date']
numerical_columns = ['selic', 'confidence', 'pib', 'incc', 'ipca', 'dolar', 'monthly_inflation', 'gdp_growth', 'dollar_growth', 'real_interest_rate', 'inflation_confidence_difference']

In [25]:
imputer = KNNImputer(n_neighbors=5)
imputed_values = imputer.fit_transform(df_macroeconomic_book_missing[numerical_columns])
df_macroeconomic_book_missing = pd.DataFrame(imputed_values, columns=numerical_columns)
df_macroeconomic_book_missing['date'] = dates
df_macroeconomic_book_missing = df_macroeconomic_book_missing[['date','selic', 'confidence', 'pib', 'incc', 'ipca', 'dolar', 'monthly_inflation', 'gdp_growth', 'dollar_growth', 'real_interest_rate', 'inflation_confidence_difference']]

* In the code above, missing data in the macroeconomic dataset stored in the "df_macroeconomic_book_missing" DataFrame was addressed. The process involved the use of the KNNImputer, an imputation method based on the k-nearest neighbors algorithm. With a parameter of n_neighbors set to 5, it considered the five nearest neighbors for each missing value in the numerical columns. The resulting imputed values are stored in the "df_macroeconomic_book_missing" DataFrame. Additionally, a 'date' column was appended to facilitate time-based analysis, ensuring that imputed values are associated with their respective dates.

In [26]:
df_macroeconomic_book_missing.isna().sum()

date                               0
selic                              0
confidence                         0
pib                                0
incc                               0
ipca                               0
dolar                              0
monthly_inflation                  0
gdp_growth                         0
dollar_growth                      0
real_interest_rate                 0
inflation_confidence_difference    0
dtype: int64

#### Save Files

In [27]:
df_macroeconomic_book = df_macroeconomic_book_missing.copy()

In [28]:
Path(file_path_book).mkdir(parents=True, exist_ok=True)
df_macroeconomic_book.to_csv(file_path_book + "/macroeconomic_book.csv", index=False)

### Stocks Book

In [29]:
df_stocks_cleaned = pd.read_csv(file_path_cleaned + "/stocks_cleaned.csv")
df_stocks_cleaned.head()

,date,ticker,adj_close,close,high,low,open,volume
0,2019-01-02,AALR3.SA,13.116832,13.250000,13.500000,13.250000,13.310000,264200.0
1,2019-01-02,ABCB4.SA,13.077145,17.120001,17.200001,16.350000,16.469999,571700.0
2,2019-01-02,ABEV3.SA,13.950426,16.150000,16.299999,15.400000,15.400000,18692900.0
3,2019-01-02,ADHM3.SA,1.243981,1.243981,1.243981,1.235687,1.235687,2170.0
4,2019-01-02,AFLT3.SA,4.572968,5.400000,5.480000,5.400000,5.480000,500.0


#### Feature Creation

In [30]:
df_stocks_book_cols = df_stocks_cleaned.copy()

In [31]:
df_stocks_book_cols['daily_return'] = df_stocks_book_cols['adj_close'].pct_change()
df_stocks_book_cols['short_term_moving_average'] = df_stocks_book_cols['adj_close'].rolling(window=10).mean()
df_stocks_book_cols['moving_average_mean'] = df_stocks_book_cols['adj_close'].rolling(window=50).mean()
df_stocks_book_cols['long_term_moving_average'] = df_stocks_book_cols['adj_close'].rolling(window=200).mean()

window = 14
delta = df_stocks_book_cols['adj_close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
average_gain = gain.rolling(window=window).mean()
average_loss = loss.rolling(window=window).mean()
rs = average_gain / average_loss
df_stocks_book_cols['rsi'] = 100 - (100 / (1 + rs))

df_stocks_book_cols['volatility'] = df_stocks_book_cols['adj_close'].pct_change().rolling(window=20).std()
df_stocks_book_cols['average_volume'] = df_stocks_book_cols['volume'].rolling(window=20).mean()
df_stocks_book_cols['daily_range'] = df_stocks_book_cols['high'] - df_stocks_book_cols['low']
df_stocks_book_cols['open_close_difference'] = df_stocks_book_cols['open'] - df_stocks_book_cols['adj_close']
df_stocks_book_cols['high_low_difference'] = df_stocks_book_cols['high'] - df_stocks_book_cols['low']
df_stocks_book_cols['acceleration'] = df_stocks_book_cols['adj_close'].pct_change().rolling(window=5).mean()
df_stocks_book_cols['hammer'] = (df_stocks_book_cols['open'] > df_stocks_book_cols['adj_close']) & (df_stocks_book_cols['low'] > df_stocks_book_cols['adj_close'] - 0.01)
df_stocks_book_cols['cagr'] = ((df_stocks_book_cols['adj_close'] / df_stocks_book_cols['adj_close'].shift(1)) - 1).mean()


In this section, we explore crucial financial metrics in our dataset:

- **daily_return**: Tracks the daily percentage change in asset prices, revealing daily performance dynamics.

- **short_term_moving_average**: Computes the average asset price over a brief time frame, aiding short-term trend analysis.

- **moving_average_mean**: Highlights the mean price over a specified time window, offering insights into overall price trends.

- **long_term_moving_average**: Reflects the average asset price over an extended duration, facilitating long-term trend assessment.

- **rsi (Relative Strength Index)**: Measures price momentum, aiding in identifying overbought or oversold conditions.

- **volatility**: Quantifies price variability, assisting in risk evaluation and market analysis.

- **average_volume**: Indicates average trading volume, valuable for assessing asset liquidity and investor interest.

- **daily_range**: Measures the price range between daily highs and lows, a critical factor for risk evaluation.

- **open_close_difference**: Calculates the difference between opening and closing prices, revealing intraday price movements.

- **high_low_difference**: Quantifies the spread between daily high and low prices, aiding in identifying price extremes.

- **acceleration**: Analyzes the rate of price change, providing insights into price movement speed.

- **hammer**: Identifies hammer candlestick patterns for potential trend reversals in technical analysis.

- **cagr (Compound Annual Growth Rate)**: Computes the annual growth rate of an investment over a specified period, valuable for assessing long-term performance.


In [37]:
df_stocks_book_cols.head(2)

,date,ticker,adj_close,close,high,low,open,volume,daily_return,short_term_moving_average,...,long_term_moving_average,rsi,volatility,average_volume,daily_range,open_close_difference,high_low_difference,acceleration,hammer,cagr
0,2019-01-02,AALR3.SA,13.116832,13.250000,13.500000,13.25,13.310000,264200.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.25,0.193169,0.25,NaN,True,2.693383
1,2019-01-02,ABCB4.SA,13.077145,17.120001,17.200001,16.35,16.469999,571700.0,-0.003026,NaN,...,NaN,NaN,NaN,NaN,0.85,3.392855,0.85,NaN,True,2.693383


#### Cleaning Missing Values

In [38]:
df_stocks_book_missing = df_stocks_book_cols.copy()

In [39]:
df_stocks_book_missing.isna().sum()

date                           0
ticker                         0
adj_close                      0
close                          0
high                           0
low                            0
open                           0
volume                         0
daily_return                   1
short_term_moving_average      9
moving_average_mean           49
long_term_moving_average     199
rsi                           13
volatility                    20
average_volume                19
daily_range                    0
open_close_difference          0
high_low_difference            0
acceleration                   5
hammer                         0
cagr                           0
dtype: int64

In [40]:
numeral_columns = df_stocks_book_missing.select_dtypes(include=['number'])
numeral_values = df_stocks_book_missing[numeral_columns.columns]

imputer = KNNImputer(n_neighbors=5)
imputed_values = imputer.fit_transform(numeral_values)
numeral_values = pd.DataFrame(imputed_values, columns=numeral_values.columns)

categorias_columns = df_stocks_book_missing.select_dtypes(exclude=['number'])
categorias_values = df_stocks_book_missing[categorias_columns.columns]

df_stocks_book_missing = pd.concat([numeral_values, categorias_values], axis=1)
df_stocks_book_missing.head(2)

,adj_close,close,high,low,open,volume,daily_return,short_term_moving_average,moving_average_mean,long_term_moving_average,...,volatility,average_volume,daily_range,open_close_difference,high_low_difference,acceleration,cagr,date,ticker,hammer
0,13.116832,13.250000,13.500000,13.25,13.310000,264200.0,-0.244840,24.336316,26.887147,40.026139,...,2.608253,1350674.45,0.25,0.193169,0.25,1.183744,2.693383,2019-01-02,AALR3.SA,True
1,13.077145,17.120001,17.200001,16.35,16.469999,571700.0,-0.003026,15.502900,16.790465,28.631415,...,4.124573,1511679.60,0.85,3.392855,0.85,2.168443,2.693383,2019-01-02,ABCB4.SA,True


* In the code above, missing data in the macroeconomic dataset stored in the "df_macroeconomic_book_missing" DataFrame was addressed. The process involved the use of the KNNImputer, an imputation method based on the k-nearest neighbors algorithm. With a parameter of n_neighbors set to 5, it considered the five nearest neighbors for each missing value in the numerical columns. The resulting imputed values are stored in the "df_stocks_book_missing" DataFrame. Additionally, a 'date' column was appended to facilitate time-based analysis, ensuring that imputed values are associated with their respective dates.

In [41]:
df_stocks_book = df_stocks_book_missing.copy()
df_stocks_book = df_stocks_book[['ticker','adj_close', 'close', 'high', 'low', 'open', 'volume', 'daily_return', 'short_term_moving_average', 'moving_average_mean', 'long_term_moving_average', 'rsi', 'volatility', 'average_volume', 'daily_range', 'open_close_difference', 'high_low_difference', 'acceleration', 'cagr', 'date', 'hammer']]

##### Save Files

In [42]:
Path(file_path_book).mkdir(parents=True, exist_ok=True)
df_stocks_book.to_csv(file_path_book + "/stocks_book.csv", index=False)